# 02 · Feature Engineering e Pipelines
## Detecção de Lavagem de Dinheiro (AML)

**Objetivo:** Construir features robustas para modelagem, com pipelines reprodutíveis e foco em prevenção de data leakage.

### Introdução

Em detecção de AML, features temporais e de rede são cruciais porque transações suspeitas frequentemente envolvem padrões sequenciais e conexões entre entidades. Escolhi implementar agregações por entidade porque dados financeiros são intrinsecamente temporais - um cliente que movimenta grandes valores em janelas curtas pode indicar comportamento de lavagem. Para evitar data leakage, todas as features são calculadas apenas com dados históricos disponíveis no momento da transação.

### Pipeline de Transformação

```
Raw Data → Cleaning → Feature Engineering → Validation → Pipeline
    ↓         ↓             ↓                ↓          ↓
- Load     - Remove     - Temporal       - Correlation - Sklearn
- Parse    - Duplicates - Aggregations   - Analysis   - Pipeline
- Anonymize - Missing   - Network        - Leakage    - Reproducible
            - Values    - Features       - Check      - Training
```

### Estratégia de Features

1. **Features Temporais**: Agregações por entidade em janelas móveis (7d, 30d)
2. **Features de Rede**: Métricas de conectividade entre contas
3. **Features Categóricas**: Encoding seguro sem leakage
4. **Features Derivadas**: Razões e taxas calculadas eticamente

> **Decisão de Design:** Priorizei features interpretáveis sobre complexas para garantir que o modelo possa ser explicado para compliance regulatória.

In [1]:
# Setup e Importações
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Adicionar src ao path
sys.path.append(str(Path('..').resolve()))

# Imports core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuração visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
# Configurar sys.path para importações
import sys
from pathlib import Path

# Adicionar diretório src ao path
sys.path.append(str(Path('..').resolve()))

# Importações das funções refatoradas
from src.features.aml_features import (
    load_raw_transactions,
    validate_data_compliance,
    clean_transactions,
    impute_and_encode,
    aggregate_by_entity,
    compute_network_features,
    create_temporal_features,
    create_network_features,
    encode_categorical_features,
    AMLFeaturePipeline
)

# Importar Pattern Feature Engineer
from src.features.pattern_engineering import PatternFeatureEngineer

# Importar função de cálculo de IV
from src.features.iv_calculator import calculate_iv

Funções locais implementadas com sucesso!


## ▸ Carregamento e Preparação dos Dados

Carregamos os dados brutos e aplicamos limpeza inicial, garantindo compliance e anonimização.

In [3]:
# Carregar dados processados
df_raw = pd.read_csv('../data/processed/transactions_enriched_final.csv')

# Garantir tipos corretos
df_raw['Timestamp'] = pd.to_datetime(df_raw['Timestamp'])
df_raw['is_fraud'] = df_raw['is_fraud'].astype(int)

print(f"Dataset: {len(df_raw):,} transações | Fraude: {df_raw['is_fraud'].mean():.3%}")

Dataset: 5,078,345 transações | Fraude: 0.102%


## ▸ Limpeza e Pré-processamento

Aplicamos limpeza para remover duplicatas, valores inválidos e garantir integridade dos dados.

In [4]:
# Limpeza básica dos dados
df_clean = df_raw.drop_duplicates()
df_clean = df_clean.dropna(subset=['Amount Paid', 'Timestamp'])
df_clean = df_clean[df_clean['Amount Paid'] > 0]

print(f"Dataset limpo: {len(df_clean):,} transações")

Dataset limpo: 5,078,336 transações


## ▸ Features Temporais

Criamos agregações por entidade em janelas temporais para capturar padrões comportamentais. Esta é a feature mais importante porque transações de lavagem frequentemente ocorrem em bursts temporais.

In [5]:
# Garantir ordenação temporal antes de criar features
df_clean_sorted = df_clean.sort_values('Timestamp').reset_index(drop=True)

# Renomear colunas para formato esperado pela função
df_clean_sorted = df_clean_sorted.rename(columns={
    'Timestamp': 'timestamp',
    'From Account': 'source',
    'Amount Paid': 'amount'
})

temporal_features_full_df = create_temporal_features(df_clean_sorted, windows=[7, 30])

print(f"Features temporais criadas: {temporal_features_full_df.shape[1]} colunas")

2025-10-19 13:17:28,137 - INFO - Creating temporal features with pandas...


[temporal] Starting temporal feature creation with pandas...
[temporal] Processing window=7 days...
[temporal] Finished window=7d in 1511.23s
[temporal] Processing window=30 days...
[temporal] Finished window=30d in 1210.69s


2025-10-19 14:02:52,949 - INFO - Created 12 temporal features using pandas


[temporal] Done. Total time: 2724.81s
Features temporais criadas: 42 colunas


In [6]:
import pandas as pd

# Combinar features temporais com dados limpos
df_with_temporal_full = pd.concat([df_clean.reset_index(drop=True), temporal_features_full_df.reset_index(drop=True)], axis=1)

# Remover duplicatas se existirem
df_with_temporal_full = df_with_temporal_full.loc[:, ~df_with_temporal_full.columns.duplicated()]

# Amostra para análise IV
df_temporal_iv_sample = df_with_temporal_full.sample(n=min(100000, len(df_with_temporal_full)), random_state=42)

# Calcular IV
iv_results_temporal = calculate_iv(df_temporal_iv_sample, target_col='is_fraud')

temporal_cols = [col for col in temporal_features_full_df.columns if 'source_amount' in col or 'hour' in col or 'day_of_week' in col or 'is_business' in col or 'is_weekend' in col]
temporal_iv_results = iv_results_temporal[iv_results_temporal['variable'].isin(temporal_cols)].copy()

temporal_iv_results.to_csv('../artifacts/temporal_features_iv_report_corrected.csv', index=False)

predictive_count = len(temporal_iv_results[temporal_iv_results['IV'] >= 0.02])
print(f"Features temporais preditivas: {predictive_count}")

Features temporais preditivas: 3


## ▸ Features de Rede

Analisamos a conectividade entre contas para identificar padrões de lavagem estruturada. Contas fraudulentas frequentemente formam clusters densos.

In [8]:
# Features de Rede
# Renomear colunas para formato esperado pela função
df_clean_network = df_clean.rename(columns={
    'From Account': 'source',
    'To Account': 'target'
})

network_features_full_df = create_network_features(df_clean_network)

print(f"Features de rede calculadas: {len(network_features_full_df)} nós")

2025-10-19 14:16:14,510 - INFO - Creating network features...
2025-10-19 14:16:36,653 - INFO - Created network features for 515080 nodes
2025-10-19 14:16:36,653 - INFO - Created network features for 515080 nodes


Features de rede calculadas: 515080 nós



In [10]:
# Combinar features de rede com dados transacionais
df_with_network = df_clean.copy()

# Renomear colunas para merge
df_with_network = df_with_network.rename(columns={
    'From Account': 'source',
    'To Account': 'target'
})

# Merge features para source accounts
df_with_network = df_with_network.merge(
    network_features_full_df[['node', 'degree', 'in_degree', 'out_degree',
                        'degree_centrality', 'in_degree_centrality', 'out_degree_centrality']],
    left_on='source', right_on='node', how='left'
).rename(columns={
    'degree': 'source_degree', 'in_degree': 'source_in_degree', 'out_degree': 'source_out_degree',
    'degree_centrality': 'source_degree_centrality', 'in_degree_centrality': 'source_in_degree_centrality',
    'out_degree_centrality': 'source_out_degree_centrality'
}).drop('node', axis=1, errors='ignore')

# Merge features para target accounts
df_with_network = df_with_network.merge(
    network_features_full_df[['node', 'degree', 'in_degree', 'out_degree',
                        'degree_centrality', 'in_degree_centrality', 'out_degree_centrality']],
    left_on='target', right_on='node', how='left'
).rename(columns={
    'degree': 'target_degree', 'in_degree': 'target_in_degree', 'out_degree': 'target_out_degree',
    'degree_centrality': 'target_degree_centrality', 'in_degree_centrality': 'target_in_degree_centrality',
    'out_degree_centrality': 'target_out_degree_centrality'
}).drop('node', axis=1, errors='ignore')

# Preencher valores ausentes
network_cols = [col for col in df_with_network.columns if col.startswith(('source_', 'target_')) and
                ('degree' in col or 'centrality' in col)]
df_with_network[network_cols] = df_with_network[network_cols].fillna(0)

# Amostra para análise IV
df_network_sample = df_with_network.sample(n=min(100000, len(df_with_network)), random_state=42)

In [11]:
# Calcular Information Value para features de rede
numeric_network_cols = [col for col in network_cols if col in df_network_sample.select_dtypes(include=[np.number]).columns]

iv_results = calculate_iv(df_network_sample, target_col='is_fraud', bins=10, max_iv=10.0)

network_iv_results = iv_results[iv_results['variable'].isin(numeric_network_cols)].copy()

network_iv_results.to_csv('../artifacts/network_features_iv_analysis.csv', index=False)

In [14]:
# Decisão Final: Features baseadas em IV
temporal_predictive = len(temporal_iv_results[temporal_iv_results['IV'] >= 0.02])
network_predictive = len(network_iv_results[network_iv_results['IV'] >= 0.02])

print(f"Features temporais: {temporal_predictive} | Features de rede: {network_predictive}")

# Criar DataFrame final com features selecionadas
df_final_features = df_clean.copy()

if temporal_predictive > 0:
    # Renomear colunas para formato esperado pela função
    df_clean_renamed = df_clean.rename(columns={
        'Timestamp': 'timestamp',
        'From Account': 'source',
        'Amount Paid': 'amount'
    })
    temporal_features_full = create_temporal_features(df_clean_renamed, windows=[7, 30])
    temporal_cols = [col for col in temporal_features_full.columns if col.startswith('source_amount_') or col.startswith('hour') or col.startswith('day_of_week') or col.startswith('is_business_hours') or col.startswith('is_weekend')]
    df_final_features = pd.merge(df_final_features, temporal_features_full[temporal_cols], left_index=True, right_index=True, how='left')

if network_predictive > 0:
    # Renomear coluna para merge
    df_final_features_renamed = df_final_features.rename(columns={'From Account': 'source'})
    df_final_features = df_final_features_renamed.merge(
        network_features_full_df[['node', 'degree', 'in_degree', 'out_degree', 'degree_centrality', 'in_degree_centrality', 'out_degree_centrality']],
        left_on='source', right_on='node', how='left'
    ).rename(columns={
        'degree': 'source_degree', 'in_degree': 'source_in_degree', 'out_degree': 'source_out_degree',
        'degree_centrality': 'source_degree_centrality', 'in_degree_centrality': 'source_in_degree_centrality', 'out_degree_centrality': 'source_out_degree_centrality'
    }).drop('node', axis=1, errors='ignore')

print(f"Dataset final: {df_final_features.shape[0]:,} linhas × {df_final_features.shape[1]} colunas")

Features temporais: 3 | Features de rede: 12



2025-10-19 14:20:38,904 - INFO - Creating temporal features with pandas...


[temporal] Starting temporal feature creation with pandas...

[temporal] Processing window=7 days...
[temporal] Processing window=7 days...
[temporal] Finished window=7d in 1457.67s
[temporal] Processing window=30 days...
[temporal] Finished window=7d in 1457.67s
[temporal] Processing window=30 days...
[temporal] Finished window=30d in 2178.65s
[temporal] Finished window=30d in 2178.65s


2025-10-19 15:21:18,246 - INFO - Created 12 temporal features using pandas


[temporal] Done. Total time: 3639.28s

Dataset final: 5,078,336 linhas × 47 colunas
Dataset final: 5,078,336 linhas × 47 colunas


## ▸ Features Categóricas

Aplicamos encoding seguro para variáveis categóricas, garantindo que não haja data leakage através de informações futuras.

### Auditoria de Encoding Categórico e Prevenção de Data Leakage

**Análise Sênior:** A função `encode_categorical_features` pode implementar *target encoding*, que usa a variável alvo (`is_fraud`) para criar as features. Aplicar este método em todo o dataset antes da separação treino/teste resulta em **data leakage**, inflando artificialmente a performance do modelo.

**Decisão Estratégica:** Para garantir a robustez do modelo, o encoding de variáveis categóricas será integrado diretamente ao **pipeline de modelagem (notebook 03)**. Lá, o encoder será "treinado" (`.fit()`) apenas com os dados de treino e depois aplicado (`.transform()`) nos dados de treino e validação/teste separadamente. Esta célula será desativada ou simplificada para refletir essa decisão.

## ▸ Validação de Features

Analisamos correlações, distribuição e possíveis problemas de leakage antes de criar o pipeline final.

In [19]:
# Combinar features e criar patterns
df_final = df_final_features.copy()

# Mapear colunas para formato esperado
column_mapping = {
    'From Bank ID': 'from_bank',
    'To Bank ID': 'to_bank',
    'Amount Received': 'amount_received',
    'Receiving Currency': 'receiving_currency',
    'Payment Currency': 'payment_currency',
    'Amount Paid': 'amount',
    'From Account': 'from_account',
    'To Account': 'to_account',
    'Timestamp': 'timestamp'
}
df_final = df_final.rename(columns=column_mapping)

# Features baseadas em patterns
pattern_engineer = PatternFeatureEngineer()
df_with_patterns = pattern_engineer.create_pattern_similarity_features(df_final.copy())

# Top 10 features por correlação
numeric_cols = df_with_patterns.select_dtypes(include=[np.number]).columns
correlations = df_with_patterns[numeric_cols].corr()['is_fraud'].abs().sort_values(ascending=False)
top_10 = correlations.head(10)

print("Top 10 features por correlação:")
for feature, corr in top_10.items():
    print(f"  {feature}: {corr:.4f}")

# Salvar dataset
output_dir = Path('../data/processed/')
output_dir.mkdir(parents=True, exist_ok=True)
df_with_patterns.to_pickle(output_dir / 'features_with_patterns.pkl')

print(f"Dataset salvo: {df_with_patterns.shape[0]:,} linhas × {df_with_patterns.shape[1]} colunas")

Loaded 3209 pattern transactions from 370 laundering attempts

Top 10 features por correlação:
  is_fraud: 1.0000
  is_weekend: 0.0162
  same_bank_transfer: 0.0108
  amount_cv_3: 0.0099
  is_business_hours: 0.0092
  fan_in_degree: 0.0088
Top 10 features por correlação:
  is_fraud: 1.0000
  is_weekend: 0.0162
  same_bank_transfer: 0.0108
  amount_cv_3: 0.0099
  is_business_hours: 0.0092
  fan_in_degree: 0.0088
  account_in_degree: 0.0086
  Bank ID_to: 0.0057
  to_bank: 0.0057
  hour_x: 0.0055
  account_in_degree: 0.0086
  Bank ID_to: 0.0057
  to_bank: 0.0057
  hour_x: 0.0055
Dataset salvo: 5,078,336 linhas × 75 colunas
Dataset salvo: 5,078,336 linhas × 75 colunas
